In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew,kurtosis
from scipy.special import cbrt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import chisquare
from matplotlib import pyplot as plt
from pandas.tools.plotting import table
import os
import pickle
from openpyxl import load_workbook,Workbook
import math

In [2]:
from advanced_analysis_package import analyze

In [3]:
os.chdir('../Input_Data')

In [4]:
df = pd.read_csv('train.csv',header=0)

In [5]:
df.columns

Index(['Symbol', 'Date', 'Series', 'Prev_Close', 'High_1d', 'Low_1d',
       'Last_1d', 'Close', 'VWAP_1d', 'Volume_1d',
       ...
       'return_4d_500', 'return_5d_500', 'Voltality_500', 'return_1d',
       'return_2d', 'return_3d', 'return_4d', 'return_5d', 'Voltality',
       'Open_1d'],
      dtype='object', length=134)

In [6]:
df_train = df.drop(['Symbol','Series','Date'],axis=1)

In [7]:
df_train.columns

Index(['Prev_Close', 'High_1d', 'Low_1d', 'Last_1d', 'Close', 'VWAP_1d',
       'Volume_1d', 'Turnover_1d', 'Trades_1d', 'Deliverable_Volume_1d',
       ...
       'return_4d_500', 'return_5d_500', 'Voltality_500', 'return_1d',
       'return_2d', 'return_3d', 'return_4d', 'return_5d', 'Voltality',
       'Open_1d'],
      dtype='object', length=131)

In [8]:
rename_dict = {col : col.replace('%','pct_') for col in df_train.columns if '%' in col}

In [9]:
rename_dict

{'%Deliverble_1d': 'pct_Deliverble_1d'}

In [10]:
df_train.rename(rename_dict,axis=1,inplace=True)

In [11]:
os.chdir('../pickles')

In [12]:
numerical,categorical = analyze.numerical_categorical_division(df_train)

In [13]:
order = {}
for col in [x for x in numerical if x != 'Close' and 'return' not in x and 'voltality' not in x.lower() and 'pct' not in x]:
    order[col] = len(str(max(df_train[col].astype(int))))

In [14]:
pickle.dump(order,open('order.pickle','wb'))

In [15]:
order

{'Prev_Close': 5,
 'High_1d': 5,
 'Low_1d': 5,
 'Last_1d': 5,
 'VWAP_1d': 5,
 'Volume_1d': 9,
 'Turnover_1d': 11,
 'Trades_1d': 7,
 'Deliverable_Volume_1d': 9,
 'Open_nifty_1d': 5,
 'High_nifty_1d': 5,
 'Low_nifty_1d': 5,
 'Close_nifty_1d': 5,
 'Volume_nifty_1d': 9,
 'Turnover_nifty_1d': 11,
 'Open_auto_1d': 5,
 'High_auto_1d': 5,
 'Low_auto_1d': 5,
 'Close_auto_1d': 5,
 'Open_bank_1d': 5,
 'High_bank_1d': 5,
 'Low_bank_1d': 5,
 'Close_bank_1d': 5,
 'Open_fmcg_1d': 5,
 'High_fmcg_1d': 5,
 'Low_fmcg_1d': 5,
 'Close_fmcg_1d': 5,
 'Open_it_1d': 5,
 'High_it_1d': 5,
 'Low_it_1d': 5,
 'Close_it_1d': 5,
 'Open_media_1d': 4,
 'High_media_1d': 4,
 'Low_media_1d': 4,
 'Close_media_1d': 4,
 'Open_metal_1d': 4,
 'High_metal_1d': 4,
 'Low_metal_1d': 4,
 'Close_metal_1d': 4,
 'Open_pharma_1d': 5,
 'High_pharma_1d': 5,
 'Low_pharma_1d': 5,
 'Close_pharma_1d': 5,
 'Open_realty_1d': 3,
 'High_realty_1d': 3,
 'Low_realty_1d': 3,
 'Close_realty_1d': 3,
 'Open_psu_1d': 4,
 'High_psu_1d': 4,
 'Low_psu_1d'

In [16]:
rename_dict={}
for col in order.keys():
    df_train[col]=df_train[col]/10**order[col]
    rename_dict[col] = col+"(in {})".format("10^{}".format(order[col]))

In [17]:
rename_dict

{'Prev_Close': 'Prev_Close(in 10^5)',
 'High_1d': 'High_1d(in 10^5)',
 'Low_1d': 'Low_1d(in 10^5)',
 'Last_1d': 'Last_1d(in 10^5)',
 'VWAP_1d': 'VWAP_1d(in 10^5)',
 'Volume_1d': 'Volume_1d(in 10^9)',
 'Turnover_1d': 'Turnover_1d(in 10^11)',
 'Trades_1d': 'Trades_1d(in 10^7)',
 'Deliverable_Volume_1d': 'Deliverable_Volume_1d(in 10^9)',
 'Open_nifty_1d': 'Open_nifty_1d(in 10^5)',
 'High_nifty_1d': 'High_nifty_1d(in 10^5)',
 'Low_nifty_1d': 'Low_nifty_1d(in 10^5)',
 'Close_nifty_1d': 'Close_nifty_1d(in 10^5)',
 'Volume_nifty_1d': 'Volume_nifty_1d(in 10^9)',
 'Turnover_nifty_1d': 'Turnover_nifty_1d(in 10^11)',
 'Open_auto_1d': 'Open_auto_1d(in 10^5)',
 'High_auto_1d': 'High_auto_1d(in 10^5)',
 'Low_auto_1d': 'Low_auto_1d(in 10^5)',
 'Close_auto_1d': 'Close_auto_1d(in 10^5)',
 'Open_bank_1d': 'Open_bank_1d(in 10^5)',
 'High_bank_1d': 'High_bank_1d(in 10^5)',
 'Low_bank_1d': 'Low_bank_1d(in 10^5)',
 'Close_bank_1d': 'Close_bank_1d(in 10^5)',
 'Open_fmcg_1d': 'Open_fmcg_1d(in 10^5)',
 'High_f

In [18]:
df_train.rename(rename_dict,axis=1,inplace=True)

In [19]:
numerical,categorical = analyze.numerical_categorical_division(df_train)

In [20]:
pickle.dump(numerical,open('numerical.pickle','wb'))
pickle.dump(categorical,open('categorical.pickle','wb'))

In [21]:
edd_df = analyze.edd(df_train,dv='Close')

In [22]:
edd_df

,Var,type,count,nmiss,missing_rate,unique,std,skewness,kurtosis,mean,...,p5,p10,p50,p90,p95,p99,mean+2sigma,mean+3sigma,max,correlation/p_value
0,Prev_Close(in 10^5),numeric,85799,0,0.0,40149,0.027475,7.094109,63.574735,0.013702,...,0.00151,0.0020525,0.0065965,0.0282402,0.038123,0.167294,0.068652,0.096128,0.328619,0.999657
1,High_1d(in 10^5),numeric,85799,0,0.0,33009,0.027833,7.093108,63.505373,0.013880,...,0.00153695,0.0020864,0.00669,0.0285796,0.0385704,0.170477,0.069546,0.097380,0.3348,0.999560
2,Low_1d(in 10^5),numeric,85799,0,0.0,34474,0.027137,7.099734,63.726008,0.013529,...,0.001488,0.0020205,0.006501,0.0279366,0.0377141,0.16508,0.067804,0.094942,0.324681,0.999564
3,Last_1d(in 10^5),numeric,85799,0,0.0,32316,0.027468,7.093875,63.576753,0.013700,...,0.00151095,0.002054,0.0065955,0.0282252,0.0381022,0.167014,0.068635,0.096102,0.32849,0.999658
4,Close,numeric,85799,0,0.0,40151,2748.678954,7.090611,63.508936,1370.756671,...,151.05,205.34,659.85,2824.43,3815.1,16754.2,6868.114579,9616.793533,32861.9,1.000000
5,VWAP_1d(in 10^5),numeric,85799,0,0.0,68264,0.027483,7.094506,63.586795,0.013706,...,0.0015111,0.00205478,0.0066003,0.0282495,0.0381505,0.167402,0.068671,0.096154,0.329752,0.999616
6,Volume_1d(in 10^9),numeric,85799,0,0.0,84982,0.005628,12.911713,434.523278,0.003206,...,5.92005e-05,0.000163064,0.00162097,0.00767185,0.0114072,0.0229038,0.014462,0.020090,0.293553,-0.174668
7,Turnover_1d(in 10^11),numeric,85799,0,0.0,85799,2394.152393,22.751982,1667.818691,1692.340335,...,100.646,228.527,1071.27,3716.94,5038.07,9335.92,6480.645122,8874.797515,248177,-0.002358
8,Trades_1d(in 10^7),numeric,85799,0,0.0,59307,0.004494,4.346556,71.941654,0.004989,...,0.00047965,0.00098946,0.0038958,0.0100729,0.0127184,0.0200867,0.013977,0.018470,0.178827,-0.137706
9,Deliverable_Volume_1d(in 10^9),numeric,85799,0,0.0,84222,0.002803,18.772913,943.429345,0.001593,...,3.06933e-05,7.8866e-05,0.000836182,0.00380612,0.00551424,0.0108717,0.007199,0.010002,0.216378,-0.169604


In [23]:
os.chdir('../Statistics')

In [24]:
wb = Workbook()
wb.save('report.xlsx')

In [25]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx')
edd_df.to_excel(writer,sheet_name='edd_v01',index=False)

In [26]:
writer.close()

In [27]:
os.chdir('../Graphs/')

In [ ]:
analyze.graphical_analysis(data=df_train,dv='Close',regression=True,path='')

In [28]:
os.chdir('../Statistics/')

In [29]:
transform_dict = {'log':lambda x: np.log(x),'sqr':lambda x: x**2,'sqrt':lambda x: np.sqrt(x),'exp':lambda x:np.exp(x),
                 'cube':lambda x: x**3,'cuberoot': lambda x: cbrt(x)}

In [30]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import skew,kurtosis
from scipy.special import cbrt

In [31]:
def transformations(data,iv,dv,cat_ratio):
    numerical,categorical = analyze.numerical_categorical_division(data)
    if iv in numerical and iv != dv:
        if len(list(data[iv].value_counts())) <= cat_ratio*data.shape[0]:
            return 'categorical'
        else:
            corr_max = np.abs(edd_df.loc[edd_df['Var']==iv,'correlation/p_value'].values)
            transformations = []
            data_new = data[[iv,dv]].dropna(how='any',axis=0)
            if edd_df.loc[edd_df['Var']==iv,'skewness'].values[0] >0:
                transform_keys = ['log','sqrt','cuberoot']
            else:
                transform_keys = ['sqr','cube','exp']
            for t in transform_keys:
                try:
                    data_new[iv] = data_new[iv].apply(transform_dict[t])
                    if data_new[data_new[iv].isin([float('inf'),-float('inf'),np.nan])].shape[0]==0:
                        corr_matrix = data_new.corr()
                        corr = corr_matrix.loc[dv,iv]
                        if np.abs(corr)>corr_max:
                            transformations.append(t)
                except Exception as e:
                    print(iv)
                    print(e)
            return ','.join(transformations)
    else:
        return None

In [32]:
edd_df['conversions'] = edd_df['Var'].apply(lambda x:transformations(df_train,x,'Close',.01))

In [33]:
edd_df

,Var,type,count,nmiss,missing_rate,unique,std,skewness,kurtosis,mean,...,p10,p50,p90,p95,p99,mean+2sigma,mean+3sigma,max,correlation/p_value,conversions
0,Prev_Close(in 10^5),numeric,85799,0,0.0,40149,0.027475,7.094109,63.574735,0.013702,...,0.0020525,0.0065965,0.0282402,0.038123,0.167294,0.068652,0.096128,0.328619,0.999657,
1,High_1d(in 10^5),numeric,85799,0,0.0,33009,0.027833,7.093108,63.505373,0.013880,...,0.0020864,0.00669,0.0285796,0.0385704,0.170477,0.069546,0.097380,0.3348,0.999560,
2,Low_1d(in 10^5),numeric,85799,0,0.0,34474,0.027137,7.099734,63.726008,0.013529,...,0.0020205,0.006501,0.0279366,0.0377141,0.16508,0.067804,0.094942,0.324681,0.999564,
3,Last_1d(in 10^5),numeric,85799,0,0.0,32316,0.027468,7.093875,63.576753,0.013700,...,0.002054,0.0065955,0.0282252,0.0381022,0.167014,0.068635,0.096102,0.32849,0.999658,
4,Close,numeric,85799,0,0.0,40151,2748.678954,7.090611,63.508936,1370.756671,...,205.34,659.85,2824.43,3815.1,16754.2,6868.114579,9616.793533,32861.9,1.000000,None
5,VWAP_1d(in 10^5),numeric,85799,0,0.0,68264,0.027483,7.094506,63.586795,0.013706,...,0.00205478,0.0066003,0.0282495,0.0381505,0.167402,0.068671,0.096154,0.329752,0.999616,
6,Volume_1d(in 10^9),numeric,85799,0,0.0,84982,0.005628,12.911713,434.523278,0.003206,...,0.000163064,0.00162097,0.00767185,0.0114072,0.0229038,0.014462,0.020090,0.293553,-0.174668,log
7,Turnover_1d(in 10^11),numeric,85799,0,0.0,85799,2394.152393,22.751982,1667.818691,1692.340335,...,228.527,1071.27,3716.94,5038.07,9335.92,6480.645122,8874.797515,248177,-0.002358,log
8,Trades_1d(in 10^7),numeric,85799,0,0.0,59307,0.004494,4.346556,71.941654,0.004989,...,0.00098946,0.0038958,0.0100729,0.0127184,0.0200867,0.013977,0.018470,0.178827,-0.137706,log
9,Deliverable_Volume_1d(in 10^9),numeric,85799,0,0.0,84222,0.002803,18.772913,943.429345,0.001593,...,7.8866e-05,0.000836182,0.00380612,0.00551424,0.0108717,0.007199,0.010002,0.216378,-0.169604,log


In [34]:
edd_df.loc[edd_df['conversions']=='categorical',['Var','unique']]

,Var,unique


In [35]:
wb = load_workbook('report.xlsx')
writer = pd.ExcelWriter('report.xlsx')
edd_df.to_excel(writer,sheet_name='edd_v01',index=False)
writer.close()

In [36]:
os.chdir('../output_data')

In [37]:
for col in numerical:
    applied = edd_df.loc[edd_df['Var']==col,'conversions'].values[0]
    if applied != '' and applied != 'categorical':
        try:
            for t in applied.split(','):
                df_train[col+'_'+t] = df_train[col].apply(transform_dict[t])
        except:
            pass

In [38]:
df_train.to_csv('train_v01.csv',index=False)

In [39]:
df_train

,Prev_Close(in 10^5),High_1d(in 10^5),Low_1d(in 10^5),Last_1d(in 10^5),Close,VWAP_1d(in 10^5),Volume_1d(in 10^9),Turnover_1d(in 10^11),Trades_1d(in 10^7),Deliverable_Volume_1d(in 10^9),...,return_5d_realty_cube,Voltality_realty_log,Voltality_psu_log,return_1d_500_sqr,return_2d_500_sqr,return_3d_500_sqr,return_4d_500_sqr,return_5d_500_sqr,Voltality_500_log,Voltality_log
0,0.001277,0.001288,0.001243,0.001270,127.30,0.001264,0.001037,131.073599,0.001166,0.000401,...,-0.922586,0.630417,0.543955,0.119906,0.001540,0.088097,4.508965,0.011694,-0.131353,1.294721
1,0.001273,0.001298,0.001262,0.001265,132.20,0.001272,0.000066,8.336131,0.000100,0.000012,...,52.264298,0.587652,0.539359,0.003077,0.119906,0.001540,0.088097,4.508965,-0.151476,0.643877
2,0.001322,0.001334,0.001260,0.001320,136.45,0.001294,0.000848,109.804318,0.001013,0.000492,...,-0.039136,0.085417,-2.136427,0.007724,0.003077,0.119906,0.001540,0.088097,-1.586059,0.664257
3,0.001364,0.001370,0.001329,0.001363,137.05,0.001357,0.000870,117.990538,0.001333,0.000476,...,-4.117022,0.768495,0.313708,3.371296,0.007724,0.003077,0.119906,0.001540,-0.370138,0.669263
4,0.001371,0.001407,0.001347,0.001357,132.65,0.001373,0.001811,248.579874,0.001645,0.000800,...,-0.025689,0.699813,0.297382,0.112121,3.371296,0.007724,0.003077,0.119906,-0.410858,0.456425
5,0.001326,0.001375,0.001312,0.001316,131.55,0.001332,0.001749,232.997317,0.001043,0.001030,...,0.113198,0.630584,0.254709,0.005445,0.112121,3.371296,0.007724,0.003077,-0.375587,0.859884
6,0.001316,0.001330,0.001296,0.001318,135.50,0.001319,0.001100,145.081642,0.001232,0.000596,...,5.006991,0.526850,0.285644,0.117966,0.005445,0.112121,3.371296,0.007724,-0.414538,0.793342
7,0.001355,0.001364,0.001307,0.001364,140.00,0.001335,0.002061,275.011099,0.001181,0.001105,...,106.879314,0.682286,0.131758,0.220120,0.117966,0.005445,0.112121,3.371296,-0.468900,0.826639
8,0.001400,0.001410,0.001350,0.001401,141.70,0.001381,0.001636,226.007351,0.001837,0.001004,...,79.091045,0.524675,-0.408629,1.690044,0.220120,0.117966,0.005445,0.112121,-0.871872,0.837819
9,0.001417,0.001438,0.001387,0.001430,149.40,0.001412,0.000891,125.798609,0.001561,0.000405,...,0.023745,0.224694,-0.089636,0.376012,1.690044,0.220120,0.117966,0.005445,-0.479972,0.836279


In [40]:
df_train.shape

(85799, 209)